In [1]:
# TODO
# 1. get total sales for all years using invoice table
# you will want to use SUBSTR to get the year from the invoice date
# you will want to use SUM to get the total sales for each year

# 2. get total sales for each country - use invoice table
# you will also need to join with the customer table - those have country info

# 3. a count tracks in each playlist - use playlist_track table
# 3. b extra challenge get total track lenght in minutes for each playlist
# you will need to join with the track table
# 3. c cherry on top - provide names of these playlists
# so you will want to join with the playlist table as well

In [2]:
import sqlite3
import pandas as pd

In [3]:
class SqliteCRUD:
    
    # constructor
    # called when we create an object of this class
    def __init__(self, dbpath):
        self.dbpath = dbpath
        self.conn = sqlite3.connect(dbpath)
        self.cursor = self.conn.cursor()
        # print status  
        print(f"Connected to database: {dbpath} Ready to perform CRUD operations!")
    
    # destructor - will be called when object is deleted - typically when program ends
    def __del__(self):
        print("Closing connection")
        self.conn.close()
    
    # get a list of all artists
    def read_artists(self):
        self.cursor.execute("SELECT * FROM artists")
        return self.cursor.fetchall() # returns a list of tuples

    def create_artist(self, name):
        # notice the parameterized query - for security reasons
        self.cursor.execute("INSERT INTO artists (name) VALUES (?)", (name,))
        self.conn.commit()
        # return self to allow chaining
        return self

    # insert a list of artists
    def create_artists(self, artists):
        self.cursor.executemany("INSERT INTO artists (name) VALUES (?)", artists)
        # alternative:
        # for artist in artists:
        #     self.cursor.execute("INSERT INTO artists (name) VALUES (?)", artist)
        # should be similar speed as long as we do not have too many artists
        # also note how we only commit once
        self.conn.commit()
        return self

    # update artist name
    def update_artist_by_id(self, id, new_name):
        # again parameterized query for security reasons
        self.cursor.execute("UPDATE artists SET name = ? WHERE ArtistId = ?", (new_name, id))
        self.conn.commit()
        return self

    # update arits name by name
    def update_artist_by_name(self, old_name, new_name):
        # again parameterized query for security reasons
        # this could change multiple rows !
        self.cursor.execute("UPDATE artists SET name = ? WHERE name = ?", (new_name, old_name))
        self.conn.commit()
        return self

    # let's get tail of artists table
    def read_artists_tail(self, n=10):
        self.cursor.execute("SELECT * FROM artists ORDER BY ArtistId DESC LIMIT ?", (n,))
        return self.cursor.fetchall()

    # delete artist by id
    def delete_artist_by_id(self, id):
        # again parameterized query for security reasons
        self.cursor.execute("DELETE FROM artists WHERE ArtistId = ?", (id,))
        self.conn.commit()
        return self

    # delete artist by name
    def delete_artist_by_name(self, name):
        # again parameterized query for security reasons
        self.cursor.execute("DELETE FROM artists WHERE name = ?", (name,))
        self.conn.commit()
        return self

    # return dataframe with all artists
    def read_artists_df(self):
        # by abstracting this we can change the implementation later
        # also we create documentation for this method
        return pd.read_sql_query("SELECT * FROM artists", self.conn)

    def read_albums_df(self):
        return pd.read_sql_query("SELECT * FROM albums", self.conn)

    def read_tracks_df(self):
        return pd.read_sql_query("SELECT * FROM tracks", self.conn)

    def read_album_stats(self):
        query = """
        --how about getting total running length of each album?
        SELECT
            t.albumid,
            a.Title AlbumTitle,
            a2.Name ArtistName,
            COUNT(trackid),
            SUM(Milliseconds)/(1000*60) AlbumLengthMinutes,
            AVG(Milliseconds)/(1000*60) AverageSongLengthMin,
            MIN(Milliseconds)/(1000*60) ShortestSongMin,
            MAX(Milliseconds)/(1000*60) LongestSongMin
        FROM
            tracks t
        JOIN albums a 
        ON a.AlbumId = t.AlbumId 
        JOIN artists a2 
        ON a.ArtistId = a2.ArtistId 
        GROUP BY
            t.albumid
        --SO HAVING is similar to WHERE but we use it in GROUP BY situations
        -- disabling HAVING it for DF since we can filter there 
        -- HAVING AlbumLengthMinutes < 80
        ORDER BY AlbumLengthMinutes DESC;
        """
        return pd.read_sql_query(query, self.conn)

In [4]:
db = SqliteCRUD("chinook.db")
db.dbpath

Connected to database: chinook.db Ready to perform CRUD operations!


'chinook.db'

In [5]:
#del db

In [6]:
artist_list = db.read_artists()
artist_list[:3]

[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith')]

In [7]:
# create a new artist
db.create_artist("David Bowie")
artist_list = db.read_artists()
# last 3 
artist_list[-3:]

[(279, 'Lauris Reinikis'), (280, 'Samanta Tīne'), (281, 'David Bowie')]

In [8]:
db.create_artist("Ansis")\
    .create_artist("Lauris Reinikis")\
    .create_artist("Samanta Tīne")\
    .read_artists_tail(8)

[(284, 'Samanta Tīne'),
 (283, 'Lauris Reinikis'),
 (282, 'Ansis'),
 (281, 'David Bowie'),
 (280, 'Samanta Tīne'),
 (279, 'Lauris Reinikis'),
 (278, 'Ansis'),
 (277, 'David Bowie')]

In [9]:
# get album stats dataframe
album_stats_df = db.read_album_stats()
album_stats_df.head()

,AlbumId,AlbumTitle,ArtistName,COUNT(trackid),AlbumLengthMinutes,AverageSongLengthMin,ShortestSongMin,LongestSongMin
0,229,"Lost, Season 3",Lost,26,1177,45.298450,42,84
1,253,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),24,1170,48.759572,47,49
2,230,"Lost, Season 1",Lost,25,1080,43.236624,40,43
3,231,"Lost, Season 2",Lost,24,1054,43.951133,41,47
4,228,"Heroes, Season 1",Heroes,23,996,43.319035,42,44


In [10]:
artists_df = db.read_artists_df()
artists_df.head()

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [11]:
# a list of single element tuples
my_artists = [("David Bowie",), ("Queen",), ("The Beatles",)]
db.create_artists(my_artists)
artist_list = db.read_artists()
# last 3
artist_list[-3:]

[(285, 'David Bowie'), (286, 'Queen'), (287, 'The Beatles')]